In [ ]:
# !pip install tensorflow-text
# !pip install tf-models-official

In [20]:
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')

In [ ]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess,
                                         name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True,
                             name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    
    net = tf.keras.layers.Dense(4, activation='sigmoid', name='classifier')(net)
    return tf.keras.Model(text_input, net)

In [4]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [22]:
df = pd.read_csv("NLP_dataset.csv")
df

,0,1
0,do sports,Physical Activity
1,doing sports,Physical Activity
2,training,Physical Activity
3,to train,Physical Activity
4,sport,Physical Activity
...,...,...
1106,practicing mindfulness,Daily Routine
1107,rearranging furniture,Daily Routine
1108,cooking meals,Daily Routine
1109,calling family or friends,Daily Routine


In [23]:
X, y = df["0"], df["1"]
y = pd.get_dummies(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=0,
                                                    shuffle=True,
                                                    stratify=y.to_numpy())

In [24]:
classifier_model = build_classifier_model()

In [25]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

In [28]:
EPOCHS = 5
steps_per_epoch = 42
num_train_steps = steps_per_epoch * EPOCHS
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [30]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [31]:
tf.config.run_functions_eagerly(True)
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(X_train, y_train,
                    epochs = EPOCHS, validation_split = 0.1,
                    verbose=1)

Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Epoch 1/5
25/25 [==============================] - 14s 382ms/step - loss: 0.6174 - binary_accuracy: 0.6546 - val_loss: 0.2995 - val_binary_accuracy: 0.9073
Epoch 2/5
25/25 [==============================] - 7s 267ms/step - loss: 0.2318 - binary_accuracy: 0.9352 - val_loss: 0.1666 - val_binary_accuracy: 0.9354
Epoch 3/5
25/25 [==============================] - 7s 298ms/step - loss: 0.1262 - binary_accuracy: 0.9615 - val_loss: 0.1231 - val_binary_accuracy: 0.9607
Epoch 4/5
25/25 [==============================] - 7s 282ms/step - loss: 0.0788 - binary_accuracy: 0.9800 - val_loss: 0.1292 - val_binary_accuracy: 0.9579
Epoch 5/5
25/25 [==============================] - 8s 306ms/step - loss: 0.0615 - binary_accuracy: 0.9828 - val_loss: 0.1096 - val_binary_accuracy: 0.9607


In [32]:
model_json = classifier_model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

classifier_model.save_weights("model.h5")